---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [101]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
#%matplotlib notebook
#import matplotlib.pyplot as plt


---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [62]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [63]:
def graph_identification():
    G = P1_Graphs[0]
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
    #plt.plot(degree_values,histogram,'o')
    #plt.xlabel('Degree')
    #plt.ylabel('Fraction of Nodes')
    #plt.xscale('log')
    #plt.yscale('log')
    #plt.show()
    #for i in range(5):
    #    G = P1_Graphs[i]
    #    print(i)
    #    print(nx.average_clustering(G))
    #    print(nx.average_shortest_path_length(G))
    return ['PA','SW_L','SW_L','PA','SW_H']
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [41]:
G = nx.read_gpickle('email_prediction.txt')


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [2]:
def salary_predictions():

    clu = nx.clustering(G)
    dis = G.degree()
    deg = nx.degree_centrality(G)
    clo = nx.closeness_centrality(G)
    btw = nx.betweenness_centrality(G,normalized = True, endpoints = False)
    op=[]
    for x in G.nodes(data = True):
        op.append(x[1])
    # Creates DataFrame. 
    df = pd.DataFrame(op)
    del df['Department']
    df['clustering'] = list(clu.values())
    df['d_distribution'] = list(dis.values())
    df['d_central'] = list(deg.values())
    df['closeness'] = list(clo.values())
    df['between'] = list(btw.values())

    nans=df[df['ManagementSalary'].isnull()]
    fill=df[df['ManagementSalary'].notnull()]
    fill
    X = fill.iloc[:,1:]
    y = fill.iloc[:,:1]
    naX = nans.iloc[:,1:]
    naY = nans.iloc[:,:1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

    dt = DecisionTreeClassifier(max_depth=4).fit(X_train, y_train)
    #y_score = dt.decision_function(X_test)
    #fpr_lr, tpr_lr = roc_curve(y_test, y_score_lr)
    #$roc_auc_lr = auc(fpr_lr, tpr_lr)

    dt2 = DecisionTreeClassifier(max_depth=4).fit(X, y)
    y_proba_dt = dt2.predict_proba(naX)
    #for a 
    #y_proba_dt
    #naX
    pred = []
    for i in y_proba_dt:
        pred.append(i[1])
    nans['proba'] = pred
    return nans['proba']


### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [93]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [94]:
def new_connections_predictions():
    jac = [x[2] for x in list(nx.jaccard_coefficient(G))]
    ra = [x[2] for x in list(nx.resource_allocation_index(G))]
    ad = [x[2] for x in list(nx.adamic_adar_index(G))]
    gh = list(nx.preferential_attachment(G))
    df = pd.DataFrame([[(x[0],x[1]),x[2]] for x in gh], columns = ['a','pref_att'])
    df.set_index('a',drop=True, inplace = True)
    del df.index.name
    df['jaccard'] = jac
    df['resource'] = ra
    df['adamic'] = ad
    df_tot = pd.merge(future_connections, df, left_index=True, right_index=True)
    df_nan = df_tot[df_tot['Future Connection'].isnull()]
    df_fill = df_tot[df_tot['Future Connection'].notnull()]
    X = df_fill.iloc[:,1:]
    y = df_fill.iloc[:,0]
    naX = df_nan.iloc[:,1:]
    dt2 = DecisionTreeClassifier(max_depth=4).fit(X, y)
    y_proba_dt = dt2.predict_proba(naX)
    df_nan['proba'] = [x[1] for x in y_proba_dt]
    return df_nan['proba']